In [1]:
import twitter

In [2]:
# initialize api instance
twitter_api = twitter.Api(consumer_key='XXXXXXXXXXX',
                        consumer_secret='XXXXXXXXXXXXXXXXXXX',
                        access_token_key='XXXXXXXXXXXXXXX',
                        access_token_secret='XXXXXXXXXXXXXXXX')

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Sun Mar 02 01:31:05 +0000 2014", "default_profile": true, "description": "A gooner who always renew his Hotstar account in hope to see my team do well in PL and also an engineer.", "favourites_count": 303, "followers_count": 18, "friends_count": 144, "id": 2367943692, "id_str": "2367943692", "name": "Ashutosh Singh", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_image_url": "http://pbs.twimg.com/profile_images/1006423447654711297/YPAh9TF9_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1006423447654711297/YPAh9TF9_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "95ashu_s", "status": {"created_at": "Tue Mar 

In [3]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [69]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:Really annoyed
Fetched 100 tweets for the term Really annoyed
[{'text': 'Whenever I read about how folks are “surprised” that women like Dr. Birx are the real power, I’m only further annoy… https://t.co/ir1gKsUgD8', 'label': None}, {'text': 'My husband made me stand here as he took this pic. I rolled my eyes and was really annoyed. I looked out the window… https://t.co/baT6Uk4DIF', 'label': None}, {'text': 'as much as elena annoyed me, i wish she didn’t leave vd): i really can’t unsee her &amp; damon together', 'label': None}, {'text': "@GeorgeC68457786 I know some of you like them, I'm glad to hear it.\nHowever between people annoyed by it and the fa… https://t.co/CvoygB03Ia", 'label': None}]


In [11]:
def buildTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time
    
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            #print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [ ]:
corpusFile = "data/corpus.csv"
tweetDataFile = "data/tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

In [21]:
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','.AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [22]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)

In [70]:
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [63]:
preprocessedTrainingSet[0:2]

[(['get', 'swype', 'iphone', 'crack', 'iphone'], 'positive'),
 (['hilarious',
   'video',
   'guy',
   'duet',
   "'s",
   'siri',
   'pretty',
   'much',
   'sums',
   'love',
   'affair'],
  'positive')]

In [71]:
preprocessedTestSet[0:2]

[(['whenever',
   'read',
   'folks',
   '“',
   'surprised',
   '”',
   'women',
   'like',
   'dr.',
   'birx',
   'real',
   'power',
   '’',
   'annoy…'],
  None),
 (['husband',
   'made',
   'stand',
   'took',
   'pic',
   'rolled',
   'eyes',
   'really',
   'annoyed',
   'looked',
   'window…'],
  None)]

In [25]:
def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [26]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

In [28]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [36]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [72]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Negative Sentiment
Negative Sentiment Percentage = 4.0%


In [76]:
NBResultLabels.count('neutral')

95

In [77]:
NBResultLabels.count('positive')

1

In [78]:
NBResultLabels.count('negative')

4